<a href="https://colab.research.google.com/github/mjgpinheiro/AI-Strategies-StockMarket/blob/master/IB_AI_Stock_Trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance


In [ ]:
!pip install ibapi

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
# Import necessary libraries
!pip install yfinance
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Define stock tickers and date range
stocks = ['AAPL', 'GOOG', 'TSLA', 'ASRT', 'PBF']
start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime(2022, 3, 7)

# Download stock data from Yahoo Finance
print("Downloading stock data...")
stock_data = yf.download(stocks, start=start_date, end=end_date)
print("Stock data downloaded successfully.")

# Create a new CSV file and write the data to it
stock_data.to_csv('stock_data.csv', index=True)
# Write the data to a CSV file
stock_data.to_csv('stock_data.csv')

# Load the data
data = pd.read_csv("stock_data.csv")

# Drop columns that are not needed
stock_data = stock_data.drop(columns=['Adj Close'])
#stock_data = stock_data.drop(columns=['Close'])
print(stock_data)
# Check if there are any missing values in the stock data
if stock_data.isnull().values.any():
    # Replace missing values with the column mean
    stock_data = stock_data.fillna(stock_data.mean())
#stock_data=stock_data[:,~np.all(np.isnan(stock_data), axis=0)]
# Normalize the data
stock_data_norm = (stock_data - stock_data.mean()) / stock_data.std()
print("Stock Norm:", stock_data_norm)

# Define the training, validation, and testing sets
##train_data = stock_data_norm.iloc[:15]
##val_data = stock_data_norm.iloc[15:20]
##test_data = stock_data_norm.iloc[20:]

#test again
train_data = stock_data_norm.iloc[:15]
val_data = stock_data_norm.iloc[0:15]
test_data = stock_data_norm.iloc[15:]

#test_data = stock_data_norm.iloc[35:,:]

print("Train data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Test data shape:", test_data.shape)

# Make sure that the training data contains at least two rows
if train_data.shape[0] < 2:
    raise ValueError("Training data must contain at least two rows.")

# Train a linear regression model on the training set
##lr = LinearRegression()
##lr.fit(X_train, y_train)

# Define the features and target variables for the testing set
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]

# Reset the index of X_test and y_test
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Train a linear regression model on the training set
lr = LinearRegression()
lr.fit(X_test, y_test)

# Use the trained model to make predictions on the testing set
y_test_pred = lr.predict(X_test)



# Use the model to make predictions on the testing set
#y_test_pred = lr.predict(X_test)
#y_test_pred = lr.predict(y_test)

# Evaluate the performance of the model using the mean squared error
test_mse = mean_squared_error(y_test, y_test_pred)
print("Testing MSE:", test_mse)

# Define take profit and stop loss percentages
take_profit_percent = 0.03
stop_loss_percent = 0.01

# Display the list of best stocks to buy and sell
buy_codes = []
sell_codes = []

for i in range(len(stocks)):
    if y_test_pred[i] > stock_data_norm.iloc[-1, i]:
        buy_codes.append(stocks[i])
    else:
        sell_codes.append(stocks[i])

print("Buy Codes:", buy_codes)
print("Sell Codes:", sell_codes)


class IBTrader(EWrapper, EClient):
    def __init__(self, host, port, client_id):
        EClient.__init__(self, self)
        self.connect(host, port, client_id)
        self.order_id = None

    def nextValidId(self, orderId):
        super().nextValidId(orderId)
        self.order_id = orderId

    def place_order(self, symbol, quantity, action):
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "STK"
        contract.currency = "USD"
        contract.exchange = "SMART"

        order = Order()
        order.action = action
        order.totalQuantity = quantity
        order.orderType = "MKT"

        self.placeOrder(self.order_id, contract, order)

    def disconnect(self):
        self.done = True
        self.disconnect()

# Define your Interactive Brokers' credentials
host = "your_host"
port = 7496 # or 7497 for paper trading
client_id = 0 # use different client IDs for different instances of the EClient class

# Create an instance of the IBTrader class
trader = IBTrader(host, port, client_id)

# Connect to the trading platform
trader.connect()

# Define a function to filter the stocks based on their P/E ratio
def filter_stocks_by_pe_ratio(stocks, max_pe_ratio):
    filtered_stocks = []
    for code in stocks:
        pe_ratio = get_pe_ratio(code)
        if pe_ratio is not None and pe_ratio <= max_pe_ratio:
            filtered_stocks.append(code)
    return filtered_stocks

# Set the maximum P/E ratio for filtering the stocks
max_pe_ratio = 20

# Filter the stocks based on their P/E ratio
filtered_stocks = filter_stocks_by_pe_ratio(stocks, max_pe_ratio)

# Submit orders for buying or selling the stocks
buy_codes = []
sell_codes = []

for i in range(len(stocks)):
    if y_test_pred[i] > stock_data_norm.iloc[-1, i]:
        buy_codes.append(stocks[i])
    else:
        sell_codes.append(stocks[i])

# Risk Management
# Set a maximum amount of capital to be allocated to each trade
max_capital_per_trade = 10000


# Sell the stocks
for code in sell_codes:
    # Get the current stock price
    current_price = get_current_price(code)
    
    # Calculate the number of shares to sell based on the maximum capital per trade
    num_shares = int(max_capital_per_trade / current_price)
    
    # Calculate the stop-loss price as a percentage of the current price
    stop_loss_percent = 0.95  # set the stop-loss percentage to 5%
    stop_loss_price = current_price * stop_loss_percent
    
    # Place the sell order
    trader.place_order(code, num_shares, "SELL")
    
    # Place a stop-loss order to limit potential losses
    stop_loss_order = Order()
    stop_loss_order.action = "SELL"
    stop_loss_order.orderType = "STP"
    stop_loss_order.totalQuantity = num_shares
    stop_loss_order.auxPrice = stop_loss_price
    
    trader.placeOrder(trader.order_id, contract, take_profit_order)
    trader.placeOrder(trader.order_id, contract, stop_loss_order)
    # Print the details of the order
    print(f"Buy order placed for {num_shares} shares of {code} at {current_price:.2f}")
    print(f"Take profit order placed for {num_shares} shares of {code} at {take_profit_price:.2f}")
    print(f"Stop loss order placed for {num_shares} shares of {code} at {stop_loss_price:.2f}")

# Buy the stocks
for code in buy_codes:
    # Get the current stock price
    current_price = get_current_price(code)
    
    # Calculate the number of shares to buy based on the maximum capital per trade
    num_shares = int(max_capital_per_trade / current_price)
    
    # Calculate the take-profit and stop-loss prices as a percentage of the current price
    take_profit_percent = 1.05  # set the take-profit percentage to 5%
    take_profit_price = current_price * take_profit_percent
    
    stop_loss_percent = 0.95  # set the stop-loss percentage to 5%
    stop_loss_price = current_price * stop_loss_percent
    
    # Place the buy order
    trader.place_order(code, num_shares, "BUY")
    
    # Place a take-profit order and a stop-loss order to limit potential gains and losses
    take_profit_order = Order()
    take_profit_order.action = "SELL"
    take_profit_order.orderType = "LMT"
    take_profit_order.totalQuantity = num_shares
    take_profit_order.lmtPrice = take_profit_price
    
    stop_loss_order = Order()
    stop_loss_order.action = "SELL"
    stop_loss_order.orderType = "STP"
    stop_loss_order.totalQuantity = num_shares
    stop_loss_order.auxPrice = stop_loss_price
    
    trader.placeOrder(trader.order_id, contract, take_profit_order)
    trader.placeOrder(trader.order_id, contract, stop_loss_order)
    
    # Print the details of the order
    print(f"Buy order placed for {num_shares} shares of {code} at {current_price:.2f}")
    print(f"Take profit order placed for {num_shares} shares of {code} at {take_profit_price:.2f}")
    print(f"Stop loss order placed for {num_shares} shares of {code} at {stop_loss_price:.2f}")


# Disconnect from the trading platform
trader.disconnect()
